# Erprobe Ziffernerkennung

In [ ]:
%%capture
%run "02 Erkennung von Ziffern.ipynb"

In [ ]:
print(f"Genauigkeit von k nächste Nachbarn:\t {neigh.score(X_test, y_test)*100:.02f} %")
print(f"Genauigkeit vom Random Forest:\t\t {clf.score(X_test, y_test)*100:.02f} %")

Bette diese Prädiktoren nun in eine Anwendung ein.

In [ ]:
import PIL
import PIL.ImageDraw
import tkinter
import numpy as np

Skaliere Bild auf 8x8 Pixel und wandle es in einen 1d-Array um, damit es als Eingabe für die ML-Algorithmen verwendet werden kann.

In [ ]:
def resize_image(pil_image):
    "resizes a PIL image to 8x8 pixels and scales the intensity values to 0..16"
    resized_digit_image = output_image.resize((8, 8), PIL.Image.LANCZOS)
    resized_digit_array = np.array(resized_digit_image)
    new_100_percent = np.partition(resized_digit_array.flatten(), -9)[-9]
    rescaled_digit_repr = (
        (resized_digit_array - resized_digit_array.min())
        / (new_100_percent - resized_digit_array.min())) * 16
    np.clip(rescaled_digit_repr, 0, 16, out=rescaled_digit_repr)
    return rescaled_digit_repr

Definiere eine kleine einfache Tkinter-Anwendung für eine Zifferneingabe.

In [ ]:
width = 200
height = 200
output_image = PIL.Image.new("L", (width, height))
center = height//2
white = (255, 255, 255)

def paint(event):
    x1, y1 = (event.x - 1), (event.y - 1)
    x2, y2 = (event.x + 1), (event.y + 1)
    canvas.create_oval(x1, y1, x2, y2, fill="white", width=5)
    draw.line([x1, y1, x2, y2], fill="white", width=5)

def run_estimator():
    global output_image, draw
    rescaled_digit_repr = resize_image(output_image)
    plt.imshow(rescaled_digit_repr)
    plt.show()
    digit_category_rf = clf.predict([rescaled_digit_repr.flatten()])
    digit_category_knn = neigh.predict([rescaled_digit_repr.flatten()])
    label = tkinter.Label(master, text=f"RF predicted: {digit_category_rf}\nKNN predicted: {digit_category_knn}")
    label.pack()
    output_image = PIL.Image.new("L", (width, height))
    draw = PIL.ImageDraw.Draw(output_image)

master = tkinter.Tk()
canvas = tkinter.Canvas(master, width=width, height=height, bg='white')
canvas.pack()
draw = PIL.ImageDraw.Draw(output_image)
canvas.pack(expand=tkinter.YES, fill=tkinter.BOTH)
canvas.bind("<B1-Motion>", paint)
button = tkinter.Button(text="predict", command=run_estimator)
button.pack()
button = tkinter.Button(text="clear", command=lambda : canvas.delete("all"))
button.pack()

Starte Tkinter-Applikation

In [ ]:
master.mainloop()